In [1]:
import re
import pandas as pd
import os

In [2]:
INPUT = 'input'
MAILDIR = 'maildir'
input_dir = os.path.join(
    os.path.abspath('..'),
    INPUT,
    MAILDIR
)
df = pd.DataFrame(columns=['user', 'folder', 'file_name', 'content'])
# iterate through the input_dir and save the first level of directories as user, the second level as folder,
# the third level as file_name, and the content of the files as content
loc_i = 0
for root, dirs, files in os.walk(input_dir):
    if files:
        folder_split = os.path.split(root)
        user_split = os.path.split(folder_split[0])
        for file in files:
            df.loc[loc_i, 'user'] = user_split[1]
            df.loc[loc_i, 'folder'] = folder_split[1]
            df.loc[loc_i, 'file_name'] = file
            df.loc[loc_i, 'content'] = os.path.join(root,file)
            loc_i += 1

# then we open each file and extract the fields from the first 15 lines (check first)